<a href="https://colab.research.google.com/github/DaKaufeeBoii/RAG-pipeline-STuTribe/blob/main/StuTribe_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries

In [ ]:
!pip install langchain langchain-community chromadb sentence-transformers transformers torch pypdf docx2txt

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:

# File Upload

In [ ]:
from google.colab import files
import os

print("Please upload all your knowledge files (.txt, .pdf, .docx):")
uploaded = files.upload() # This will capture all uploaded files

if not uploaded:
    print("\nNo files uploaded. Please run this cell again.")
else:
    print(f"\nSuccessfully uploaded {len(uploaded)} file(s):")
    for filename in uploaded.keys():
        print(f"- {filename}")

Please upload all your knowledge files (.txt, .pdf, .docx):


Saving TensorFlow-for-Machine-Intelligence-A-Hands-On-Introduction-to-Learning-Algorithms.pdf to TensorFlow-for-Machine-Intelligence-A-Hands-On-Introduction-to-Learning-Algorithms.pdf

Successfully uploaded 1 file(s):
- TensorFlow-for-Machine-Intelligence-A-Hands-On-Introduction-to-Learning-Algorithms.pdf


# RAG pipeline (go to "LEVEL 3")

In [ ]:
# Cell 2: Write the app.py file
%%writefile app.py

import streamlit as st
import warnings
import os
import textwrap
import torch
import asyncio
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from collections import deque

# --- Langchain Imports ---
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# --- Suppress warnings ---
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# ==============================================================================
# 2. CACHED FUNCTIONS (For loading models and processing files)
# ==============================================================================

@st.cache_resource
def get_embeddings_model():
    """Loads and caches the embeddings model."""
    print("Loading embeddings model...")
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

@st.cache_resource
def get_reranker_model():
    """Loads and caches the reranker model."""
    print("Loading reranker model...")
    return HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")

@st.cache_resource
def get_llm():
    """Loads and caches the LLM pipeline."""
    print("Loading LLM model (flan-t5-large)...")
    model_id = "google/flan-t5-large"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline(
        "text2text-generation", model=model, tokenizer=tokenizer,
        max_length=512, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    return HuggingFacePipeline(pipeline=pipe)

@st.cache_resource
def create_rag_chain(_file_data_tuple, llm, embeddings, reranker_model):
    """
    Loads, splits, and processes files to create a cached RAG chain.
    """
    print("Creating RAG chain... (Processing files)")
    all_documents = []
    temp_files_to_clean = []

    for filename, file_content in _file_data_tuple:
        try:
            with open(filename, "wb") as f:
                f.write(file_content)
            temp_files_to_clean.append(filename)
        except Exception as e:
            print(f"ERROR writing temp file {filename}: {e}")
            continue

        file_extension = os.path.splitext(filename)[1].lower()
        loader = None
        if file_extension == '.pdf':
            loader = PyPDFLoader(filename)
        elif file_extension == '.docx':
            loader = Docx2txtLoader(filename)
        elif file_extension == '.txt':
            loader = TextLoader(filename)
        else:
            print(f"Skipping {filename}: Unsupported file type")
            continue

        try:
            documents = loader.load()
            all_documents.extend(documents)
            print(f"Loaded {len(documents)} documents from {filename}.")
        except Exception as e:
            print(f"ERROR loading {filename}: {e}")

    # Clean up temporary files
    for temp_file in temp_files_to_clean:
        try:
            os.remove(temp_file)
        except OSError as e:
            print(f"Error cleaning up file {temp_file}: {e}")

    if not all_documents:
        st.error("No text could be extracted from the uploaded file(s).")
        return None

    # --- Split ---
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(all_documents)
    print(f"Split into {len(docs)} document chunks.")

    # --- Vector DB ---
    vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)

    # --- Retriever ---
    base_retriever = vectordb.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10}
    )
    compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    # --- Prompts ---
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

    prompt_template_str = """Use the following pieces of context to answer the question at the end.
If the answer is not contained within the context provided, just say "I do not have enough information to answer this question." Do not try to make up an answer.

{context}

Question: {question}

Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template_str, input_variables=["context", "question"]
    )

    # --- RAG Chain ---
    rag_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=compression_retriever,
        return_source_documents=True,
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

    print("RAG chain created successfully.")
    return rag_chain

# ==============================================================================
# 3. STREAMLIT APP UI
# ==============================================================================

async def run_app():
    st.set_page_config(page_title="Chat with Documents", layout="wide")
    st.title("📄 Chat with your Documents (Colab Edition)")

    # --- 1. Sidebar for File Upload ---
    st.sidebar.title("Configuration")

    uploaded_files = st.sidebar.file_uploader(
        "Upload your documents (PDF, DOCX, TXT)",
        type=["pdf", "docx", "txt"],
        accept_multiple_files=True
    )

    st.sidebar.divider()
    show_sources = st.sidebar.toggle("Show Sources", value=False)
    FAILURE_MESSAGE = "I do not have enough information to answer this question."

    # --- 2. Initialize Session State (Local, non-persistent) ---
    if "messages" not in st.session_state:
        st.session_state.messages = [] # For chat display
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = deque(maxlen=3) # For RAG chain
    if "rag_chain" not in st.session_state:
        st.session_state.rag_chain = None

    # --- 3. Process Files and Build RAG Chain ---
    if uploaded_files:
        # Create a hashable tuple of file data for caching
        file_data_tuple = tuple((f.name, f.getvalue()) for f in uploaded_files)

        # Load models first
        with st.spinner("Loading models... (This happens once)"):
            embeddings = get_embeddings_model()
            llm = get_llm()
            reranker = get_reranker_model()

        # Build chain (will be cached based on file_data_tuple)
        with st.spinner("Processing documents... This may take a while."):
            st.session_state.rag_chain = create_rag_chain(file_data_tuple, llm, embeddings, reranker)

        if st.session_state.rag_chain:
            st.sidebar.success(f"Processed {len(uploaded_files)} file(s). Ready to chat!")
        else:
            st.sidebar.error("Failed to process documents.")
            st.stop()

    elif st.session_state.rag_chain is None:
        st.info("Please upload your documents in the sidebar to start.")
        st.stop()

    # --- 4. Display Chat History ---
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if message["role"] == "assistant" and "sources" in message:
                if show_sources and message["content"] != FAILURE_MESSAGE:
                    with st.expander("Show Sources"):
                        for i, source in enumerate(message["sources"]):
                            st.info(f"Source {i+1} [Score: {source['score']:.4f}] [File: {source['file']}]\n\n"
                                    f"...{source['content']}...")

    # --- 5. Handle Chat Input ---
    prompt = st.chat_input("Ask a question about your documents:")

    if prompt:
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                rag_chain = st.session_state.rag_chain
                chat_history_list = list(st.session_state.chat_history)

                # Run the RAG chain
                result = await rag_chain.ainvoke({
                    "question": prompt,
                    "chat_history": chat_history_list
                })
                answer = result['answer'].strip()

                # --- SAVE & UPDATE STATE (Local Session) ---
                st.session_state.chat_history.append((prompt, answer))

                # Format sources for display
                sources_data = []
                if result['source_documents']:
                    for source in result['source_documents']:
                        sources_data.append({
                            "score": source.metadata.get('relevance_score', 0),
                            "file": source.metadata.get('source', 'Unknown'),
                            "content": source.page_content
                        })

                st.markdown(answer)
                st.session_state.messages.append({
                    "role": "assistant",
                    "content": answer,
                    "sources": sources_data
                })

        # Rerun to show the new messages
        st.rerun()

if __name__ == "__main__":
    # Streamlit runs in an async-friendly environment
    asyncio.run(run_app())

# Level 3 (don't run this)

In [ ]:
import warnings
import os
import textwrap
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# --- Level 2 Imports for Re-ranking ---
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# --- PRE-RUN CHECK ---
# This check is conceptual for a script. In a notebook, 'uploaded' would
# be defined in a prior cell.
if 'uploaded' not in locals() or not locals().get('uploaded'):
    print("-------------------------------------------------------------------")
    print("ERROR: No files found. Please run the file upload cell first.")
    print(" (This script assumes 'uploaded' is a dictionary like {'filename.pdf': b'...'}")
    print("-------------------------------------------------------------------")
else:
    warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
    print("RAG Pipeline: Level 2 - With Cross-Encoder Re-ranking")
    print("--------------------------------------------------")

    # --- 1. Load & Split (Looping over ALL files) ---
    print(f"Step 1: Loading and splitting {len(uploaded)} file(s)...")
    all_documents = []

    # In a real script, 'uploaded' would be file paths or content.
    # For this example, we'll assume 'uploaded' is a dictionary
    # of filenames to file content, and we'll write temp files.

    temp_files_to_clean = []

    for filename, file_content in uploaded.items():
        print(f"   - Loading {filename}...")

        # Write content to a temporary file to be loaded
        try:
            with open(filename, "wb") as f:
                f.write(file_content)
            temp_files_to_clean.append(filename)
        except Exception as e:
            print(f"     !!!! ERROR writing temp file {filename}: {e} !!!!")
            continue

        file_extension = os.path.splitext(filename)[1].lower()

        # Requirement 1: Support multiple document types (PDF, TXT, DOCX)
        if file_extension == '.pdf':
            loader = PyPDFLoader(filename)
        elif file_extension == '.docx':
            loader = Docx2txtLoader(filename)
        elif file_extension == '.txt':
            loader = TextLoader(filename)
        else:
            print(f"     !!!! Skipping {filename}: Unsupported file type {file_extension} !!!!")
            continue

        try:
            documents = loader.load()
            all_documents.extend(documents)
            print(f"     Loaded {len(documents)} documents from {filename}.")
        except Exception as e:
            print(f"     !!!! ERROR loading {filename}: {e} !!!!")

    if not all_documents:
        print("ERROR: No text could be extracted from the uploaded file(s).")
    else:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(all_documents)
        print(f"Loaded and split a total of {len(docs)} document chunks from all files.")

        # --- 2. Text Embeddings & Re-ranker Model ---
        print("Step 2: Initializing embeddings model and re-ranker...")
        # Model for initial search
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
        embeddings = HuggingFaceEmbeddings(model_name=model_name)

        # Requirement 2: Implement re-ranking
        print("   - Loading Cross-Encoder model (for re-ranking)...")
        reranker_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
        reranker_model = HuggingFaceCrossEncoder(model_name=reranker_model_name)
        print("Models loaded successfully.")

        # --- 3. Vector Database ---
        print("Step 3: Creating and populating vector database...")
        vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)
        print("Vector database created successfully.")

        # --- 4. Retrieval (Level 2) + Answer Generation ---
        print("Step 4: Setting up Level 2 Retriever and Generation Model...")

        # --- Part A: The Generation Model (LLM) ---
        model_id = "google/flan-t5-large"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
        pipe = pipeline(
            "text2text-generation", model=model, tokenizer=tokenizer,
            max_length=512, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        )
        llm = HuggingFacePipeline(pipeline=pipe)

        # --- Part B: The "Smart" Retriever (with Re-ranking) ---

        # 1. Base Retriever: Fetches 10 diverse docs using MMR
        base_retriever = vectordb.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 10} # Get 10 docs to re-rank
        )

        # 2. Re-ranker: Uses the Cross-Encoder to find the true top 3
        #    This fulfills Requirement 2
        compressor = CrossEncoderReranker(model=reranker_model, top_n=3)

        # 3. Compression Retriever: Ties them together
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )

        # --- Part C: The RAG Chain (with Custom Prompt) ---
        prompt_template_str = """Use the following pieces of context to answer the question at the end.
If the answer is not contained within the context provided, just say "I do not have enough information to answer this question." Do not try to make up an answer.

{context}

Question: {question}

Answer:"""
        PROMPT = PromptTemplate(
            template=prompt_template_str, input_variables=["context", "question"]
        )
        FAILURE_MESSAGE = "I do not have enough information to answer this question."
        chain_type_kwargs = {"prompt": PROMPT}

        rag_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=compression_retriever, # Using the new smart retriever
            return_source_documents=True,
            chain_type_kwargs=chain_type_kwargs
        )

        print("Setup complete. You can now ask questions about ALL your documents.")
        print("---------------------------------------------")
        print(f"Retriever Type: MMR (k=10) + Cross-Encoder Re-ranking (top_n=3)")
        print("Type 'sources on' or 'sources off' to toggle source visibility.")
        print("Type 'exit' to quit.")
        print("---------------------------------------------")

        # --- 5. Interactive Q&A Loop (with Confidence Score) ---
        show_sources = False
        try:
            while True:
                query = input("Ask a question: ")
                query_lower = query.lower().strip()

                if query_lower == 'exit':
                    print("Exiting...")
                    break

                if query_lower == 'sources on':
                    show_sources = True
                    print("\n[Source visibility ON]\n")
                    continue

                if query_lower == 'sources off':
                    show_sources = False
                    print("\n[Source visibility OFF]\n")
                    continue

                if not query.strip():
                    continue

                # 1. Invoke the RAG chain
                result = rag_chain.invoke(query)
                answer = result['result'].strip()

                # 2. Print the answer
                print("\nAnswer:")
                print(textwrap.fill(answer, width=80))

                # Requirement 3: Return confidence scores
                # The 'relevance_score' is added by the CrossEncoderReranker
                if answer != FAILURE_MESSAGE and result['source_documents']:
                    top_doc = result['source_documents'][0]
                    confidence = top_doc.metadata.get('relevance_score')
                    if confidence is not None:
                        # Cross-encoder scores aren't 0-1, but higher is better.
                        print(f"\nConfidence (Top Source Score): {confidence:.4f}")

                # 4. Print Sources (if toggled on)
                if show_sources and answer != FAILURE_MESSAGE and result['source_documents']:
                    print("\n--- Top Sources (Re-ranked) ---")
                    for i, source in enumerate(result['source_documents']):
                        # Also showing the confidence score for each source
                        score = source.metadata.get('relevance_score', 0)
                        print(f"Source {i+1} [Score: {score:.4f}] [File: {source.metadata.get('source', 'Unknown')}]")
                        print(textwrap.fill(f"   ...{source.page_content}...",
                                             width=80,
                                             initial_indent="   ",
                                             subsequent_indent="   "))
                    print("-------------------------------\n")
                else:
                    print("\n")
        except EOFError:
            print("\nExiting...")
        finally:
            # Clean up temporary files
            for temp_file in temp_files_to_clean:
                try:
                    os.remove(temp_file)
                    print(f"Cleaned up temp file: {temp_file}")
                except OSError as e:
                    print(f"Error cleaning up file {temp_file}: {e}")

# Level 3... **Again** (don't run this either)

In [ ]:
import warnings
import os
import textwrap
import torch
import asyncio
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from collections import deque

# --- Langchain Imports ---
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# --- Firebase Imports (for Topic Tracking) ---
import firebase_admin
from firebase_admin import credentials, firestore, auth

# --- Helper Function: Find Related Topic (Task 2, Level 2) ---
async def find_related_topic(db, collection_path, embeddings_model, new_query):
    """
    Searches all past conversations for a semantically related topic.
    """
    print("[TopicTracker] Searching past conversations...")
    try:
        col_ref = db.collection(collection_path)
        snapshot_docs = [doc async for doc in await col_ref.get()]

        all_q_and_metadata = []
        if not snapshot_docs:
            print("[TopicTracker] No past conversations found.")
            return (None, None)

        for doc_snap in snapshot_docs:
            topic_id = doc_snap.id
            history = doc_snap.to_dict().get("history", [])
            for message in history:
                all_q_and_metadata.append(
                    (message['q'], {"topic_id": topic_id, "answer": message['a']})
                )

        if not all_q_and_metadata:
            print("[TopicTracker] No past messages found to compare.")
            return (None, None)

        texts = [item[0] for item in all_q_and_metadata]
        metadata = [item[1] for item in all_q_and_metadata]

        history_vectordb = Chroma.from_texts(
            texts,
            embeddings_model,
            metadatas=metadata
        )

        results = history_vectordb.similarity_search_with_relevance_scores(
            new_query,
            k=1
        )

        if results and results[0][1] > 0.75: # 0.75 similarity threshold
            top_doc, score = results[0]
            found_topic_id = top_doc.metadata['topic_id']
            related_q = top_doc.page_content
            related_a = top_doc.metadata['answer']

            print(f"\nThis seems related to a past topic (Score: {score:.2f}):")
            print(f"  Q: {related_q}")
            print(f"  A: {related_a}")
            choice = input("  Continue this topic? (y/n): ").strip().lower()

            if choice == 'y':
                topic_doc_ref = db.collection(collection_path).document(found_topic_id)
                topic_doc = await topic_doc_ref.get()
                if topic_doc.exists:
                    full_history = topic_doc.to_dict().get("history", [])
                else:
                    full_history = []

                new_history_deque = deque(maxlen=3)
                new_history_tuples = [(msg['q'], msg['a']) for msg in full_history]
                new_history_deque.extend(new_history_tuples) # Deque handles maxlen

                print(f"[TopicTracker] Resuming topic {found_topic_id}.")
                return found_topic_id, new_history_deque

    except Exception as e:
        print(f"[TopicTracker] Error searching topics: {e}")

    return (None, None) # No topic found or user declined

# --- Helper Function: Save Conversation (Task 2, Level 2) ---
async def save_conversation(db, collection_path, topic_id, query, answer):
    """
    Saves the new Q/A pair to the appropriate topic in Firestore.
    """
    new_message = {"q": query, "a": answer}

    try:
        if topic_id:
            doc_ref = db.collection(collection_path).document(topic_id)
            await doc_ref.update({
                "history": firestore.ArrayUnion([new_message])
            })
            return topic_id
        else:
            col_ref = db.collection(collection_path)
            doc_ref = await col_ref.add({
                "history": [new_message]
            })
            print(f"[TopicTracker] Started new topic: {doc_ref.id}")
            return doc_ref.id
    except Exception as e:
        print(f"[TopicTracker] Error saving conversation: {e}")
        return topic_id # Return old ID on failure

# --- Main RAG Pipeline ---
async def main_pipeline():
    # --- PRE-RUN CHECK ---
    if 'uploaded' not in locals() or not locals().get('uploaded'):
        print("-------------------------------------------------------------------")
        print("ERROR: No files found. Please run the file upload cell first.")
        print(" (This script assumes 'uploaded' is a dictionary like {'filename.pdf': b'...'}")
        print("-------------------------------------------------------------------")
        return

    warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
    print("RAG Pipeline: Task 1 (L2) + Task 2 (L2 - Topic Tracking)")
    print("--------------------------------------------------")

    # --- Firebase/Database Setup (Task 2, Level 2) ---
    print("Step 0: Initializing Firebase and Database...")

    appId = globals().get('__app_id', 'default-app-id')
    firebase_config_str = globals().get('__firebase_config', '{}')
    firebaseConfig = json.loads(firebase_config_str)
    initial_auth_token = globals().get('__initial_auth_token')

    try:
        if not firebase_admin._apps:
            cred = credentials.ApplicationDefault()
            app = firebase_admin.initialize_app(cred, firebaseConfig)
        else:
            app = firebase_admin.get_app()

        auth_client = auth.get_auth(app)
        db = firestore.client(app)

        userId = "anonymous_user"

        if userId and userId != "anonymous_user":
             convo_collection_path = f"artifacts/{appId}/users/{userId}/conversations"
        else:
             convo_collection_path = f"artifacts/{appId}/conversations"

        print(f"Firebase App Initialized OK.")
        print(f"Database Path: {convo_collection_path}")

    except Exception as e:
        print(f"!!!! ERROR initializing Firebase: {e} !!!!")
        print("Topic tracking and history will not work.")
        db = None
        convo_collection_path = None
        userId = "offline-user"

    # --- 1. Load & Split (Looping over ALL files) ---
    print(f"Step 1: Loading and splitting {len(uploaded)} file(s)...")
    all_documents = []
    temp_files_to_clean = []

    for filename, file_content in uploaded.items():
        print(f"   - Loading {filename}...")
        try:
            if isinstance(file_content, bytes):
                 with open(filename, "wb") as f:
                    f.write(file_content)
            else:
                 with open(filename, "w", encoding='utf-8') as f:
                    f.write(file_content)

            temp_files_to_clean.append(filename)
        except Exception as e:
            print(f"     !!!! ERROR writing temp file {filename}: {e} !!!!")
            continue

        file_extension = os.path.splitext(filename)[1].lower()

        if file_extension == '.pdf':
            loader = PyPDFLoader(filename)
        elif file_extension == '.docx':
            loader = Docx2txtLoader(filename)
        elif file_extension == '.txt':
            loader = TextLoader(filename)
        else:
            print(f"     !!!! Skipping {filename}: Unsupported file type {file_extension} !!!!")
            continue

        try:
            documents = loader.load()
            all_documents.extend(documents)
            print(f"     Loaded {len(documents)} documents from {filename}.")
        except Exception as e:
            print(f"     !!!! ERROR loading {filename}: {e} !!!!")

    if not all_documents:
        print("ERROR: No text could be extracted from the uploaded file(s).")
        for temp_file in temp_files_to_clean:
            try:
                os.remove(temp_file)
            except OSError as e:
                print(f"Error cleaning up file {temp_file}: {e}")
        return

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(all_documents)
    print(f"Loaded and split a total of {len(docs)} document chunks from all files.")

    # --- 2. Text Embeddings & Re-ranker Model ---
    print("Step 2: Initializing embeddings model and re-ranker...")
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    print("   - Loading Cross-Encoder model (for re-ranking)...")
    reranker_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    reranker_model = HuggingFaceCrossEncoder(model_name=reranker_model_name)
    print("Models loaded successfully.")

    # --- 3. Vector Database (For Documents) ---
    print("Step 3: Creating and populating document vector database...")
    vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)
    print("Vector database created successfully.")

    # --- 4. Retrieval (Level 2) + Answer Generation ---
    print("Step 4: Setting up Level 2 Retriever and Generation Model...")

    # --- Part A: The Generation Model (LLM) ---
    model_id = "google/flan-t5-large"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline(
        "text2text-generation", model=model, tokenizer=tokenizer,
        max_length=512, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    # --- Part B: The "Smart" Retriever (with Re-ranking) ---
    base_retriever = vectordb.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10}
    )
    compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    # --- Part C: The RAG Chain (with Custom Prompt) ---
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

    prompt_template_str = """Use the following pieces of context to answer the question at the end.
If the answer is not contained within the context provided, just say "I do not have enough information to answer this question." Do not try to make up an answer.

{context}

Question: {question}

Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template_str, input_variables=["context", "question"]
    )
    FAILURE_MESSAGE = "I do not have enough information to answer this question."

    rag_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=compression_retriever,
        return_source_documents=True,
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

    print("Setup complete. You can now ask questions about ALL your documents.")
    print("---------------------------------------------")
    print(f"Chain Type: ConversationalRetrievalChain")
    print(f"Retriever Type: MMR (k=10) + Cross-Encoder Re-ranking (top_n=3)")
    print(f"Memory: Remembers last 3 Q/A pairs (per session)")
    print(f"Topic Tracking: Enabled (via Firestore)")
    print("Type 'sources on' or 'sources off' to toggle source visibility.")
    print("Type 'exit' to quit.")
    print("---------------------------------------------")

    # --- 5. Interactive Q&A Loop (with Confidence Score) ---
    show_sources = False

    chat_history = deque(maxlen=3)
    current_topic_id = None

    try:
        while True:
            query = input("Ask a question: ")
            query_lower = query.lower().strip()

            if query_lower == 'exit':
                print("Exiting...")
                break
            if query_lower == 'sources on':
                show_sources = True
                print("\n[Source visibility ON]\n")
                continue
            if query_lower == 'sources off':
                show_sources = False
                print("\n[Source visibility OFF]\n")
                continue
            if not query.strip():
                continue

            # --- TOPIC TRACKING LOGIC ---
            is_first_message = len(chat_history) == 0

            if is_first_message and db:
                found_id, loaded_history = await find_related_topic(
                    db, convo_collection_path, embeddings, query
                )
                if found_id and loaded_history:
                    current_topic_id = found_id
                    chat_history = loaded_history
                else:
                    print("[TopicTracker] Starting new conversation topic.")
                    current_topic_id = None
                    chat_history.clear()

            print(f"\n[Debug: Sending {len(chat_history)} items to history...]")

            result = await rag_chain.ainvoke({
                "question": query,
                "chat_history": list(chat_history)
            })

            answer = result['answer'].strip()

            # --- HISTORY & DATABASE UPDATE ---
            if db:
                new_topic_id = await save_conversation(
                    db, convo_collection_path, current_topic_id, query, answer
                )
                current_topic_id = new_topic_id

            chat_history.append((query, answer))

            # 2. Print the answer
            print("\nAnswer:")
            print(textwrap.fill(answer, width=80))

            # 3. Print Confidence Score
            if answer != FAILURE_MESSAGE and result['source_documents']:
                top_doc = result['source_documents'][0]
                confidence = top_doc.metadata.get('relevance_score')
                if confidence is not None:
                    print(f"\nConfidence (Top Source Score): {confidence:.4f}")

            # 4. Print Sources
            if show_sources and answer != FAILURE_MESSAGE and result['source_documents']:
                print("\n--- Top Sources (Re-ranked) ---")
                for i, source in enumerate(result['source_documents']):
                    score = source.metadata.get('relevance_score', 0)
                    print(f"Source {i+1} [Score: {score:.4f}] [File: {source.metadata.get('source', 'Unknown')}]")
                    print(textwrap.fill(f"   ...{source.page_content}...",
                                         width=80,
                                         initial_indent="   ",
                                         subsequent_indent="   "))
                print("-------------------------------\n")
            else:
                print("\n")
    except EOFError:
        print("\nExiting...")
    finally:
        for temp_file in temp_files_to_clean:
            try:
                os.remove(temp_file)
                print(f"Cleaned up temp file: {temp_file}")
            except OSError as e:
                print(f"Error cleaning up file {temp_file}: {e}")

# Run the main async function using await
await main_pipeline()

# LEVEL 3 (This code works)

In [ ]:
import warnings
import os
import textwrap
import torch
import asyncio
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from collections import deque

# --- Langchain Imports ---
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
# Corrected import path
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# --- Firebase Imports (for Topic Tracking) ---
try:
    import firebase_admin
    from firebase_admin import credentials, firestore
except ImportError:
    print("-------------------------------------------------------------------")
    print("ERROR: 'firebase-admin' library not found.")
    print("Please install it by running: pip install firebase-admin")
    print("-------------------------------------------------------------------")
    # This will stop the script if firebase-admin isn't installed
    raise

# --- Helper Function: Find Related Topic (Task 2, Level 2) ---
# [FIX 2: Restored asyncio.to_thread for all sync firebase-admin calls]
async def find_related_topic(db, collection_path, embeddings_model, new_query):
    """
    Searches all past conversations for a semantically related topic.
    Uses asyncio.to_thread to run synchronous firebase-admin calls.
    """
    print("[TopicTracker] Searching past conversations...")
    try:
        col_ref = db.collection(collection_path)
        # Run the synchronous .get() call in a separate thread
        snapshot = await asyncio.to_thread(col_ref.get)

        all_q_and_metadata = []
        if not snapshot: # snapshot is a list, check if it's empty
            print("[TopicTracker] No past conversations found.")
            return (None, None)

        # 1. Aggregate all past questions from all topics
        for doc_snap in snapshot: # Iterate over the list of documents
            topic_id = doc_snap.id
            history = doc_snap.to_dict().get("history", [])
            for message in history:
                all_q_and_metadata.append(
                    (message['q'], {"topic_id": topic_id, "answer": message['a']})
                )

        if not all_q_and_metadata:
            print("[TopicTracker] No past messages found to compare.")
            return (None, None)

        # 2. Create a temporary in-memory vector store of past questions
        texts = [item[0] for item in all_q_and_metadata]
        metadata = [item[1] for item in all_q_and_metadata]

        history_vectordb = Chroma.from_texts(
            texts,
            embeddings_model,
            metadatas=metadata
        )

        # 3. Search for the most similar past question
        results = history_vectordb.similarity_search_with_relevance_scores(
            new_query,
            k=1
        )

        # 4. Check if the match is good enough
        if results and results[0][1] > 0.75: # 0.75 similarity threshold
            top_doc, score = results[0]
            found_topic_id = top_doc.metadata['topic_id']
            related_q = top_doc.page_content
            related_a = top_doc.metadata['answer']

            print(f"\nThis seems related to a past topic (Score: {score:.2f}):")
            print(f"  Q: {related_q}")
            print(f"  A: {related_a}")
            choice = input("  Continue this topic? (y/n): ").strip().lower()

            if choice == 'y':
                # Load the full history for this topic
                doc_ref = db.collection(collection_path).document(found_topic_id)
                # Run sync .get() in a thread
                topic_doc = await asyncio.to_thread(doc_ref.get)

                full_history = topic_doc.to_dict().get("history", [])

                # Create a new deque and load it
                new_history_deque = deque(maxlen=3)
                new_history_tuples = [(msg['q'], msg['a']) for msg in full_history]
                new_history_deque.extend(new_history_tuples) # Deque handles maxlen

                print(f"[TopicTracker] Resuming topic {found_topic_id}.")
                return found_topic_id, new_history_deque

    except Exception as e:
        print(f"[TopicTracker] Error searching topics: {e}")

    return (None, None) # No topic found or user declined

# --- Helper Function: Save Conversation (Task 2, Level 2) ---
# [FIX 2: Restored asyncio.to_thread for all sync firebase-admin calls]
async def save_conversation(db, collection_path, topic_id, query, answer):
    """
    Saves the new Q/A pair to the appropriate topic in Firestore.
    Uses asyncio.to_thread to run synchronous firebase-admin calls.
    """
    new_message = {"q": query, "a": answer}

    try:
        if topic_id:
            # Add to an existing topic
            doc_ref = db.collection(collection_path).document(topic_id)
            # Use firestore.ArrayUnion and run .update() in a thread
            await asyncio.to_thread(
                doc_ref.update,
                {"history": firestore.ArrayUnion([new_message])}
            )
            return topic_id
        else:
            # Create a new topic
            col_ref = db.collection(collection_path)
            # Run .add() in a thread
            # .add() returns a tuple (timestamp, document_reference)
            timestamp, doc_ref = await asyncio.to_thread(
                col_ref.add,
                {"history": [new_message]}
            )
            print(f"[TopicTracker] Started new topic: {doc_ref.id}")
            return doc_ref.id
    except Exception as e:
        print(f"[TopicTracker] Error saving conversation: {e}")
        return topic_id # Return old ID on failure

# --- Main RAG Pipeline ---
async def main_pipeline():
    # --- PRE-RUN CHECK ---
    # [FIX 1: Changed 'locals()' to 'globals()' to find the 'uploaded' variable]
    if 'uploaded' not in globals() or not globals().get('uploaded'):
        print("-------------------------------------------------------------------")
        print("ERROR: 'uploaded' variable not found in global scope.")
        print("Please run the file upload cell first.")
        print(" (This script assumes 'uploaded' is a dictionary like {'filename.pdf': b'...'}")
        print("-------------------------------------------------------------------")
        return

    warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
    print("RAG Pipeline: Task 1 (L2) + Task 2 (L2 - Topic Tracking)")
    print("--------------------------------------------------")

    # --- Firebase/Database Setup (Task 2, Level 2) ---
    # [FIX 3: Restored stable Colab init method]
    print("Step 0: Initializing Firebase (firebase-admin)...")

    try:
        # Check if app is already initialized
        if not firebase_admin._apps:
            # Initialize using a service account key
            cred_path = "serviceAccountKey.json"
            if not os.path.exists(cred_path):
                raise FileNotFoundError(f"'{cred_path}' not found. Please upload it.")

            cred = credentials.Certificate(cred_path)
            firebase_admin.initialize_app(cred)

        db = firestore.client()

        # Get a User ID to separate conversations
        userId = input("Enter a unique User ID (to save/load your topics): ").strip()
        if not userId:
            userId = "default-user"
            print("No User ID entered, using 'default-user'.")

        print(f"Firebase Admin OK. Using User ID: {userId}")

        # Define the collection path for this user's private conversations
        convo_collection_path = f"colab_rag_users/{userId}/conversations"
        print(f"Database Path: {convo_collection_path}")

    except FileNotFoundError as e:
        print(f"!!!! ERROR: {e} !!!!")
        print("Please upload your Firebase Service Account Key to this Colab session.")
        print("Topic tracking and history will be disabled.")
        db = None
        convo_collection_path = None
    except Exception as e:
        print(f"!!!! ERROR initializing Firebase: {e} !!!!")
        print(f"Details: {e}")
        print("Topic tracking and history will be disabled.")
        db = None
        convo_collection_path = None

    # --- 1. Load & Split (Looping over ALL files) ---
    print(f"Step 1: Loading and splitting {len(uploaded)} file(s)...")
    all_documents = []
    temp_files_to_clean = []

    # Use globals().get() to safely access 'uploaded'
    for filename, file_content in globals().get('uploaded', {}).items():
        print(f"   - Loading {filename}...")
        try:
            # Write content (bytes or str) to a temporary file
            if isinstance(file_content, bytes):
                with open(filename, "wb") as f:
                    f.write(file_content)
            else:
                with open(filename, "w", encoding='utf-8') as f:
                    f.write(str(file_content)) # Ensure it's a string

            temp_files_to_clean.append(filename)
        except Exception as e:
            print(f"     !!!! ERROR writing temp file {filename}: {e} !!!!")
            continue

        file_extension = os.path.splitext(filename)[1].lower()

        if file_extension == '.pdf':
            loader = PyPDFLoader(filename)
        elif file_extension == '.docx':
            loader = Docx2txtLoader(filename)
        elif file_extension == '.txt':
            loader = TextLoader(filename)
        else:
            print(f"     !!!! Skipping {filename}: Unsupported file type {file_extension} !!!!")
            continue

        try:
            documents = loader.load()
            all_documents.extend(documents)
            print(f"     Loaded {len(documents)} documents from {filename}.")
        except Exception as e:
            print(f"     !!!! ERROR loading {filename}: {e} !!!!")

    if not all_documents:
        print("ERROR: No text could be extracted from the uploaded file(s).")
        # Clean up even on failure
        for temp_file in temp_files_to_clean:
            try:
                os.remove(temp_file)
            except OSError as e:
                print(f"Error cleaning up file {temp_file}: {e}")
        return

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(all_documents)
    print(f"Loaded and split a total of {len(docs)} document chunks from all files.")

    # --- 2. Text Embeddings & Re-ranker Model ---
    print("Step 2: Initializing embeddings model and re-ranker...")
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    print("   - Loading Cross-Encoder model (for re-ranking)...")
    reranker_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    reranker_model = HuggingFaceCrossEncoder(model_name=reranker_model_name)
    print("Models loaded successfully.")

    # --- 3. Vector Database (For Documents) ---
    print("Step 3: Creating and populating document vector database...")
    vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)
    print("Vector database created successfully.")

    # --- 4. Retrieval (Level 2) + Answer Generation ---
    print("Step 4: Setting up Level 2 Retriever and Generation Model...")

    # --- Part A: The Generation Model (LLM) ---
    model_id = "google/flan-t5-large"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline(
        "text2text-generation", model=model, tokenizer=tokenizer,
        max_length=512, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    # --- Part B: The "Smart" Retriever (with Re-ranking) ---
    base_retriever = vectordb.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10}
    )
    compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    # --- Part C: The RAG Chain (with Custom Prompt) ---
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

    prompt_template_str = """Use the following pieces of context to answer the question at the end.
If the answer is not contained within the context provided, just say "I do not have enough information to answer this question." Do not try to make up an answer.

{context}

Question: {question}

Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template_str, input_variables=["context", "question"]
    )
    FAILURE_MESSAGE = "I do not have enough information to answer this question."

    rag_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=compression_retriever,
        return_source_documents=True,
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

    print("Setup complete. You can now ask questions about ALL your documents.")
    print("---------------------------------------------")
    print(f"Chain Type: ConversationalRetrievalChain")
    print(f"Retriever Type: MMR (k=10) + Cross-Encoder Re-ranking (top_n=3)")
    print(f"Memory: Remembers last 3 Q/A pairs (per session)")
    if db:
        print(f"Topic Tracking: Enabled (via Firestore)")
    else:
        print(f"Topic Tracking: DISABLED (Firebase init failed)")
    print("Type 'sources on' or 'sources off' to toggle source visibility.")
    print("Type 'exit' to quit.")
    print("---------------------------------------------")

    # --- 5. Interactive Q&A Loop (with Confidence Score) ---
    show_sources = False

    # This deque holds the history for the *current session*
    chat_history = deque(maxlen=3)
    # This ID tracks the current topic in Firestore
    current_topic_id = None

    try:
        while True:
            query = input("Ask a question: ")
            query_lower = query.lower().strip()

            if query_lower == 'exit':
                print("Exiting...")
                break
            if query_lower == 'sources on':
                show_sources = True
                print("\n[Source visibility ON]\n")
                continue
            if query_lower == 'sources off':
                show_sources = False
                print("\n[Source visibility OFF]\n")
                continue
            if not query.strip():
                continue

            # --- TOPIC TRACKING LOGIC ---
            is_first_message = len(chat_history) == 0

            # Only search for topics if it's the first message of a session
            # and the database is connected
            if is_first_message and db:
                found_id, loaded_history = await find_related_topic(
                    db, convo_collection_path, embeddings, query
                )
                if found_id and loaded_history:
                    current_topic_id = found_id
                    chat_history = loaded_history # This is the loaded deque
                else:
                    print("[TopicTracker] Starting new conversation topic.")
                    current_topic_id = None # Ensure it's a new topic
                    chat_history.clear() # Ensure history is empty

            print(f"\n[Debug: Sending {len(chat_history)} items to history...]")

            # Use ainvoke for async chain call
            result = await rag_chain.ainvoke({
                "question": query,
                "chat_history": list(chat_history) # Pass history as a list
            })

            answer = result['answer'].strip()

            # --- HISTORY & DATABASE UPDATE ---
            # Save to Firestore database (if connected)
            if db:
                new_topic_id = await save_conversation(
                    db, convo_collection_path, current_topic_id, query, answer
                )
                current_topic_id = new_topic_id # Update topic ID

            # Add to local deque for next turn
            chat_history.append((query, answer))

            # 2. Print the answer
            print("\nAnswer:")
            print(textwrap.fill(answer, width=80))

            # 3. Print Confidence Score
            if answer != FAILURE_MESSAGE and result['source_documents']:
                top_doc = result['source_documents'][0]
                confidence = top_doc.metadata.get('relevance_score')
                if confidence is not None:
                    print(f"\nConfidence (Top Source Score): {confidence:.4f}")

            # 4. Print Sources
            if show_sources and answer != FAILURE_MESSAGE and result['source_documents']:
                print("\n--- Top Sources (Re-ranked) ---")
                for i, source in enumerate(result['source_documents']):
                    score = source.metadata.get('relevance_score', 0)
                    print(f"Source {i+1} [Score: {score:.4f}] [File: {source.metadata.get('source', 'Unknown')}]")
                    print(textwrap.fill(f"   ...{source.page_content}...",
                                         width=80,
                                         initial_indent="   ",
                                         subsequent_indent="   "))
                print("-------------------------------\n")
            else:
                print("\n")
    except EOFError:
        print("\nExiting...")
    finally:
        # Clean up temporary files
        for temp_file in temp_files_to_clean:
            try:
                os.remove(temp_file)
                print(f"Cleaned up temp file: {temp_file}")
            except OSError as e:
                print(f"Error cleaning up file {temp_file}: {e}")

# --- Run the main async function ---
# This is intended to be run in a notebook cell that supports top-level await
await main_pipeline()



RAG Pipeline: Task 1 (L2) + Task 2 (L2 - Topic Tracking)
--------------------------------------------------
Step 0: Initializing Firebase (firebase-admin)...
!!!! ERROR: 'serviceAccountKey.json' not found. Please upload it. !!!!
Please upload your Firebase Service Account Key to this Colab session.
Topic tracking and history will be disabled.
Step 1: Loading and splitting 1 file(s)...
   - Loading TensorFlow-for-Machine-Intelligence-A-Hands-On-Introduction-to-Learning-Algorithms.pdf...
     Loaded 305 documents from TensorFlow-for-Machine-Intelligence-A-Hands-On-Introduction-to-Learning-Algorithms.pdf.
Loaded and split a total of 1080 document chunks from all files.
Step 2: Initializing embeddings model and re-ranker...


/tmp/ipython-input-1212704918.py:265: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda
/tmp/ipython-input-1212704918.py:287: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Setup complete. You can now ask questions about ALL your documents.
---------------------------------------------
Chain Type: ConversationalRetrievalChain
Retriever Type: MMR (k=10) + Cross-Encoder Re-ranking (top_n=3)
Memory: Remembers last 3 Q/A pairs (per session)
Topic Tracking: DISABLED (Firebase init failed)
Type 'sources on' or 'sources off' to toggle source visibility.
Type 'exit' to quit.
---------------------------------------------
Ask a question: tensorflow with code example

[Debug: Sending 0 items to history...]

Answer:
I do not have enough information to answer the question.


Ask a question: what is tensorflow?

[Debug: Sending 1 items to history...]

Answer:
an open source software library for numerical computation using data flow graphs




# Streamlit Website YAY!!

installing dependencies

In [ ]:
# Cell 1: Install Dependencies
!pip install streamlit "langchain[community]" "langchain[llms]" transformers "sentence-transformers>=2.2.0" torch "PyPDF2" docx2txt "chromadb" pyngrok

Upload Files

In [ ]:
# Cell 2: Write the app.py file

import streamlit as st
import warnings
import os
import textwrap
import torch
import asyncio
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from collections import deque

# --- Langchain Imports ---
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

# --- Suppress warnings ---
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# ==============================================================================
# 2. CACHED FUNCTIONS (For loading models and processing files)
# ==============================================================================

@st.cache_resource
def get_embeddings_model():
    """Loads and caches the embeddings model."""
    print("Loading embeddings model...")
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

@st.cache_resource
def get_reranker_model():
    """Loads and caches the reranker model."""
    print("Loading reranker model...")
    return HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")

@st.cache_resource
def get_llm():
    """Loads and caches the LLM pipeline."""
    print("Loading LLM model (flan-t5-large)...")
    model_id = "google/flan-t5-large"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline(
        "text2text-generation", model=model, tokenizer=tokenizer,
        max_length=512, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    )
    return HuggingFacePipeline(pipeline=pipe)

@st.cache_resource
def create_rag_chain(_file_data_tuple, llm, embeddings, reranker_model):
    """
    Loads, splits, and processes files to create a cached RAG chain.
    """
    print("Creating RAG chain... (Processing files)")
    all_documents = []
    temp_files_to_clean = []

    for filename, file_content in _file_data_tuple:
        try:
            with open(filename, "wb") as f:
                f.write(file_content)
            temp_files_to_clean.append(filename)
        except Exception as e:
            print(f"ERROR writing temp file {filename}: {e}")
            continue

        file_extension = os.path.splitext(filename)[1].lower()
        loader = None
        if file_extension == '.pdf':
            loader = PyPDFLoader(filename)
        elif file_extension == '.docx':
            loader = Docx2txtLoader(filename)
        elif file_extension == '.txt':
            loader = TextLoader(filename)
        else:
            print(f"Skipping {filename}: Unsupported file type")
            continue

        try:
            documents = loader.load()
            all_documents.extend(documents)
            print(f"Loaded {len(documents)} documents from {filename}.")
        except Exception as e:
            print(f"ERROR loading {filename}: {e}")

    # Clean up temporary files
    for temp_file in temp_files_to_clean:
        try:
            os.remove(temp_file)
        except OSError as e:
            print(f"Error cleaning up file {temp_file}: {e}")

    if not all_documents:
        st.error("No text could be extracted from the uploaded file(s).")
        return None

    # --- Split ---
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(all_documents)
    print(f"Split into {len(docs)} document chunks.")

    # --- Vector DB ---
    vectordb = Chroma.from_documents(documents=docs, embedding=embeddings)

    # --- Retriever ---
    base_retriever = vectordb.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10}
    )
    compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    # --- Prompts ---
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

    prompt_template_str = """Use the following pieces of context to answer the question at the end.
If the answer is not contained within the context provided, just say "I do not have enough information to answer this question." Do not try to make up an answer.

{context}

Question: {question}

Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template_str, input_variables=["context", "question"]
    )

    # --- RAG Chain ---
    rag_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=compression_retriever,
        return_source_documents=True,
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )

    print("RAG chain created successfully.")
    return rag_chain

# ==============================================================================
# 3. STREAMLIT APP UI
# ==============================================================================

async def run_app():
    st.set_page_config(page_title="Chat with Documents", layout="wide")
    st.title("📄 Chat with your Documents (Colab Edition)")

    # --- 1. Sidebar for File Upload ---
    st.sidebar.title("Configuration")

    uploaded_files = st.sidebar.file_uploader(
        "Upload your documents (PDF, DOCX, TXT)",
        type=["pdf", "docx", "txt"],
        accept_multiple_files=True
    )

    st.sidebar.divider()
    show_sources = st.sidebar.toggle("Show Sources", value=False)
    FAILURE_MESSAGE = "I do not have enough information to answer this question."

    # --- 2. Initialize Session State (Local, non-persistent) ---
    if "messages" not in st.session_state:
        st.session_state.messages = [] # For chat display
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = deque(maxlen=3) # For RAG chain
    if "rag_chain" not in st.session_state:
        st.session_state.rag_chain = None

    # --- 3. Process Files and Build RAG Chain ---
    if uploaded_files:
        # Create a hashable tuple of file data for caching
        file_data_tuple = tuple((f.name, f.getvalue()) for f in uploaded_files)

        # Load models first
        with st.spinner("Loading models... (This happens once)"):
            embeddings = get_embeddings_model()
            llm = get_llm()
            reranker = get_reranker_model()

        # Build chain (will be cached based on file_data_tuple)
        with st.spinner("Processing documents... This may take a while."):
            st.session_state.rag_chain = create_rag_chain(file_data_tuple, llm, embeddings, reranker)

        if st.session_state.rag_chain:
            st.sidebar.success(f"Processed {len(uploaded_files)} file(s). Ready to chat!")
        else:
            st.sidebar.error("Failed to process documents.")
            st.stop()

    elif st.session_state.rag_chain is None:
        st.info("Please upload your documents in the sidebar to start.")
        st.stop()

    # --- 4. Display Chat History ---
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if message["role"] == "assistant" and "sources" in message:
                if show_sources and message["content"] != FAILURE_MESSAGE:
                    with st.expander("Show Sources"):
                        for i, source in enumerate(message["sources"]):
                            st.info(f"Source {i+1} [Score: {source['score']:.4f}] [File: {source['file']}]\n\n"
                                    f"...{source['content']}...")

    # --- 5. Handle Chat Input ---
    prompt = st.chat_input("Ask a question about your documents:")

    if prompt:
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                rag_chain = st.session_state.rag_chain
                chat_history_list = list(st.session_state.chat_history)

                # Run the RAG chain
                result = await rag_chain.ainvoke({
                    "question": prompt,
                    "chat_history": chat_history_list
                })
                answer = result['answer'].strip()

                # --- SAVE & UPDATE STATE (Local Session) ---
                st.session_state.chat_history.append((prompt, answer))

                # Format sources for display
                sources_data = []
                if result['source_documents']:
                    for source in result['source_documents']:
                        sources_data.append({
                            "score": source.metadata.get('relevance_score', 0),
                            "file": source.metadata.get('source', 'Unknown'),
                            "content": source.page_content
                        })

                st.markdown(answer)
                st.session_state.messages.append({
                    "role": "assistant",
                    "content": answer,
                    "sources": sources_data
                })

        # Rerun to show the new messages
        st.rerun()

if __name__ == "__main__":
    # Streamlit runs in an async-friendly environment
    asyncio.run(run_app())

Run the RAG Pipeline

In [ ]:
# Cell 3: Run the app and create the tunnel
import time
from pyngrok import ngrok

# Terminate any existing ngrok tunnels
ngrok.kill()

# Start streamlit in the background
!nohup streamlit run app.py &

# Wait for the server to start
time.sleep(5)

# Open a tunnel to the streamlit port (8501)
public_url = ngrok.connect(8501)
print("================================================================================")
print("Your Streamlit app is live!")
print(f"Click this link to open: {public_url}")
print("================================================================================")